#Data preprocessing

In [47]:
#Libraries
import pandas as pd
from google.colab import drive
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy

In [23]:
#Load data from my drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Specify the path of the CSV file in my drive
file_path = '/content/drive/My Drive/nlp_data/sample.csv'
#Read the CSV file into a DataFrame
data = pd.read_csv(file_path)

In [25]:
#Show the Data
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [26]:
#La normalisation en minuscules de la colonne "text"
data["text"] = data["text"].str.lower()
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@applesupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 your business means a lot to us. pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 i really hope you all change but i'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 livechat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@virgintrains see attached error message. i've...,119243,119240.0


In [27]:
#Suppression de la Ponctuation
import string

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation


data['text_without_punctuation'] = data['text'].apply(remove_punctuation)
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_without_punctuation
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@applesupport causing the reply to be disregar...,119236,NaN,applesupport causing the reply to be disregard...
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 your business means a lot to us. pleas...,NaN,119239.0,105835 your business means a lot to us please ...
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 i really hope you all change but i'm su...,119238,NaN,76328 i really hope you all change but im sure...
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 livechat is online at the moment - htt...,119241,119242.0,105836 livechat is online at the moment https...
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@virgintrains see attached error message. i've...,119243,119240.0,virgintrains see attached error message ive tr...


In [28]:
#Télécharger la liste des stopwords et télécharger les données nécessaires pour utiliser le tokenizer (tokeniseur) de NLTK
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
#Suppression stopWords

def remove_stopwords(text):

    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)


# Applique la fonction remove_stopwords à la colonne "text"
data['text_without_stopwords'] = data['text_without_punctuation'].apply(remove_stopwords)

# Afficher le résultat
print(data[['text_without_punctuation', 'text_without_stopwords']])


                             text_without_punctuation  \
0   applesupport causing the reply to be disregard...   
1   105835 your business means a lot to us please ...   
2   76328 i really hope you all change but im sure...   
3   105836 livechat is online at the moment  https...   
4   virgintrains see attached error message ive tr...   
..                                                ...   
88  105860 i wish amazon had an option of where i ...   
89  they reschedule my shit for tomorrow httpstcor...   
90  105861 hey sara sorry to hear of the issues yo...   
91  tesco bit of both  finding the layout cumberso...   
92  105861 if that doesnt help please dm your full...   

                               text_without_stopwords  
0   applesupport causing reply disregarded tapped ...  
1   105835 business means lot us please dm name zi...  
2          76328 really hope change im sure wont dont  
3   105836 livechat online moment httpstcosy94vtu8...  
4   virgintrains see attached error

In [32]:
#Tokenization
def tokenize_text(text):

    tokens = word_tokenize(text)
    return tokens

data['tokens'] = data['text_without_stopwords'].apply(tokenize_text)

# Afficher le résultat
print(data[['text_without_stopwords', 'tokens']])

                               text_without_stopwords  \
0   applesupport causing reply disregarded tapped ...   
1   105835 business means lot us please dm name zi...   
2          76328 really hope change im sure wont dont   
3   105836 livechat online moment httpstcosy94vtu8...   
4   virgintrains see attached error message ive tr...   
..                                                ...   
88  105860 wish amazon option get shipped ups stor...   
89        reschedule shit tomorrow httpstcorsvzct982t   
90  105861 hey sara sorry hear issues ask lay spee...   
91  tesco bit finding layout cumbersome removing i...   
92  105861 doesnt help please dm full name address...   

                                               tokens  
0   [applesupport, causing, reply, disregarded, ta...  
1   [105835, business, means, lot, us, please, dm,...  
2   [76328, really, hope, change, im, sure, wont, ...  
3   [105836, livechat, online, moment, httpstcosy9...  
4   [virgintrains, see, attached, e

In [39]:
#stemming

def perform_stemming(tokens):

    ps = PorterStemmer()
    stemmed_text = ' '.join([ps.stem(word) for word in tokens])
    return stemmed_text

data['stemmed_tokens'] = data['tokens'].apply(perform_stemming)

#Afficher le résultat

print(data[['tokens', 'stemmed_tokens']])


                                               tokens  \
0   [applesupport, causing, reply, disregarded, ta...   
1   [105835, business, means, lot, us, please, dm,...   
2   [76328, really, hope, change, im, sure, wont, ...   
3   [105836, livechat, online, moment, httpstcosy9...   
4   [virgintrains, see, attached, error, message, ...   
..                                                ...   
88  [105860, wish, amazon, option, get, shipped, u...   
89   [reschedule, shit, tomorrow, httpstcorsvzct982t]   
90  [105861, hey, sara, sorry, hear, issues, ask, ...   
91  [tesco, bit, finding, layout, cumbersome, remo...   
92  [105861, doesnt, help, please, dm, full, name,...   

                                       stemmed_tokens  
0   applesupport caus repli disregard tap notif ke...  
1   105835 busi mean lot us pleas dm name zip code...  
2           76328 realli hope chang im sure wont dont  
3   105836 livechat onlin moment httpstcosy94vtu8k...  
4   virgintrain see attach error me

In [41]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [53]:
#Lemmatisation
nlp = spacy.load('en_core_web_sm')


# Define a function for lemmatization
def lemmatize_words(text):
    doc = nlp(" ".join(text))
    return [token.lemma_ for token in doc]


data['lemmatized_tokens'] = data['tokens'].apply(lambda text: lemmatize_words(text))

# Afficher le résultat
print(data[['tokens', 'lemmatized_tokens']])



                                               tokens  \
0   [applesupport, causing, reply, disregarded, ta...   
1   [105835, business, means, lot, us, please, dm,...   
2   [76328, really, hope, change, im, sure, wont, ...   
3   [105836, livechat, online, moment, httpstcosy9...   
4   [virgintrains, see, attached, error, message, ...   
..                                                ...   
88  [105860, wish, amazon, option, get, shipped, u...   
89   [reschedule, shit, tomorrow, httpstcorsvzct982t]   
90  [105861, hey, sara, sorry, hear, issues, ask, ...   
91  [tesco, bit, finding, layout, cumbersome, remo...   
92  [105861, doesnt, help, please, dm, full, name,...   

                                    lemmatized_tokens  
0   [applesupport, cause, reply, disregard, tap, n...  
1   [105835, business, mean, lot, we, please, dm, ...  
2   [76328, really, hope, change, I, m, sure, will...  
3   [105836, livechat, online, moment, httpstcosy9...  
4   [virgintrain, see, attach, erro